In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from datetime import datetime as dt

'''
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from yellowbrick.cluster import InterclusterDistance
'''

'\nimport seaborn as sns\nfrom mpl_toolkits.mplot3d import Axes3D\nfrom scipy.cluster.hierarchy import dendrogram, linkage\nfrom sklearn.cluster import KMeans\nfrom yellowbrick.cluster import InterclusterDistance\n'

In [3]:
data = pd.read_csv('bank_transactions.csv', delimiter = ',')

In [500]:
data.columns

Index(['TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmount (INR)'],
      dtype='object')

### Formatting DateTime

In [4]:
# cleaning data
# dropping na, reformatting date times

data.dropna(inplace=True)#? doesnt change
data.CustomerDOB = pd.to_datetime(data.CustomerDOB) 
data.TransactionDate = pd.to_datetime(data.TransactionDate) 
data.TransactionTime = data.TransactionTime.apply(lambda x: dt.strptime(str(x).zfill(6), '%H%M%S').time() )

### Offset the people born after 2000 and drop the Customer born before 1900

In [5]:
#TODO add plots to justify the choise made
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)
data.drop(data[data.CustomerDOB.dt.year < 1900].index, axis=0, inplace=True)

#  Getting your data + feature engineering

In [523]:
data.columns

Index(['TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmount (INR)'],
      dtype='object')

### Number of transaction per CustomerID

In [605]:
# nr_trans
nr_trans = data[['CustomerID','TransactionID']].groupby(by = 'CustomerID').count()
nr_trans.rename(columns = {'TransactionID' : 'nr_trans'}, inplace = True)

### Number of transaction that have a value larger than 100

In [18]:
data.rename(columns = {"TransactionAmount (INR)":"TransactionAmount_INR"}, inplace = True)
nr_trans_100p = data[['CustomerID','TransactionAmount_INR']].query("TransactionAmount_INR > 100").groupby(by= "CustomerID").count().reset_index()

#Rename the column TransactionAmount_INR
nr_trans_100p.rename(columns= {"TransactionAmount_INR":"trans_greater_100"}, inplace = True)

### Mean of transactions for each customer

In [21]:
#  mean_trans 
mean_trans = data[['CustomerID','TransactionAmount_INR']].groupby(by='CustomerID').mean()
mean_trans.rename(columns = {'TransactionAmount_INR':'mean_trans' }, inplace = True)

### Mean of total Balance for each customerID

In [592]:
# mean_balance
mean_balance = data[['CustomerID','CustAccountBalance']].groupby(by='CustomerID').mean()
mean_balance.rename(columns = {'CustAccountBalance':'mean_balance' }, inplace = True)

### Mean difference between the mean balance and the mean transaction

In [603]:
#mean_diff. Aka Utilization
mean_diff = pd.DataFrame( mean_balance.mean_balance.sub(mean_trans.mean_trans, axis = 0) )
mean_diff.columns = ['mean_diff']

### Most frequent gender for each customerID

In [ ]:
#TODO: Check later :)

In [23]:
# gender
gender_most_freq = data[['CustomerID', 'CustGender']].groupby(by = 'CustomerID').apply(lambda x:x.value_counts().index[0])
gender_most_freq.rename(columns = {'CustGender' :'gender_most_freq'}, inplace = True)

KeyboardInterrupt: 

### Most frequent location 

In [609]:
# most_freq_loc
most_freq_loc = data[['CustomerID','CustLocation']].groupby(by='CustomerID').apply(lambda x: x.value_counts().index[0])
most_freq_loc.rename(columns = {'CustLocation' :'most_freq_loc'}, inplace = True)

Merge all the additional feature into one df

In [669]:
dfs = [nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc]

In [686]:
df_7 = pd.concat([nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc], 1)

/var/folders/c0/1q90pl8n3k50gnjpfmzf_lzh0000gn/T/ipykernel_24066/2719202539.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_7 = pd.concat([nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc], 1)


In [688]:
df_7.to_csv('seven_params.csv', sep=',', encoding='utf-8', index = False)

## 20 features
grouping by CustomerID then working on groups, 
stored in twenty_params.csv

In [776]:
from tqdm import tqdm

In [779]:
# TODO: check later
for i, el in tqdm( enumerate(groups) ): # the i is only here because it had to restart and didn't manage to slice groups
    if i>340000:
        name, group = el
        features_20(name, group)

839081it [38:04, 367.36it/s]  


In [62]:
def findMinMaxTrans(data) -> pd.DataFrame:
    return data.groupby("CustomerID").agg(  min_trans = ("TransactionAmount_INR", min),
                                            max_trans = ("TransactionAmount_INR", max),
                                            ).reset_index()

In [55]:
min_Trans.head()

,CustomerID,min_trans,max_trans
0,C1010011,356.0,4750.0
1,C1010012,1499.0,1499.0
2,C1010014,250.0,1205.0
3,C1010018,30.0,30.0
4,C1010024,5000.0,5000.0


In [759]:
feat_20 = {}
def features_20(name, group):
    
    #min and max of transaction for each customerID
    min_max = findMinMaxTrans(data[["CustomerID", "TransactionAmount_INR"]])
    
    quant_75_trans = group['TransactionAmount (INR)'].quantile(0.75) #?
    quant_25_trans = group['TransactionAmount (INR)'].quantile(0.25) #?
    trans_amount = group['TransactionAmount (INR)']
    trans_val_nr = trans_amount.nunique() / len(trans_amount)
    
    
    max_balance = group['CustAccountBalance'].max()
    min_balance = group['CustAccountBalance'].min()
    #std_balance = group['CustAccountBalance'].std()
    diff_balance = max_balance - min_balance

    
    times = group['TransactionTime']
    hours = pd.to_datetime(times.astype(str)).dt.hour
    # most_freq_hour = max(set(hours), key=hours.count)
    most_freq_hour = max(set(hours), key=list(hours).count)
    max_time = hours.max()
    min_time = hours.min()
    mean_time = hours.mean()
    
    date = pd.to_datetime(group.TransactionDate.astype(str)).dt
    most_freq_weekday = max(set(date.weekday), key=list(date.weekday).count )
    most_freq_month = max(set(date.month), key=list(date.month).count)
    nr_diff_months = len( set(date.month) )
    
    diff_locs = group['CustLocation'].nunique()
    
    diff_dob = group['CustomerDOB'].nunique() # ? different people
    age = group.TransactionDate.sub(group.CustomerDOB, axis = 0).mean().days/365
    age_mean = int(age)
    
    trans_amount = group['TransactionAmount (INR)']
    nr_trans_1kp = trans_amount[trans_amount>1000].count()
    
    balance_amount = group['CustAccountBalance']
    nr_trans_10kp = balance_amount[balance_amount>10000].count()
    
    trans_balance_ratio = group['TransactionAmount (INR)']/group['CustAccountBalance']
    tb_ratio_mean = trans_balance_ratio.mean()
    tb_ratio_max = trans_balance_ratio.max()


    #avg_nr_trans_perday = group.groupby(by= 'TransactionDate').count().mean()
    #first_freq_loc = group['CustLocation'].agg(lambda x:x.value_counts().index[0])
    
    feat_20[name] = [min_trans, max_trans, quant_75_trans, quant_25_trans, trans_val_nr, max_balance, min_balance, diff_balance, most_freq_hour, max_time, min_time, mean_time, most_freq_weekday, most_freq_month, nr_diff_months, diff_locs, diff_dob, age_mean, nr_trans_1kp, nr_trans_10kp, tb_ratio_mean, tb_ratio_max]
    
    

In [781]:
len(feat_20) # nr groups

839081

In [801]:
data_20 = pd.DataFrame.from_dict(feat_20, orient = 'index')

In [802]:
col_list = 'CustomerID, min_trans, max_trans, quant_75_trans, quant_25_trans, trans_val_nr, max_balance, min_balance, diff_balance, most_freq_hour, max_time, min_time, mean_time, most_freq_weekday, most_freq_month, nr_diff_months, diff_locs, diff_dob, age_mean, nr_trans_1kp, nr_trans_10kp, tb_ratio_mean, tb_ratio_max'
col_list_str = col_list.split(', ')

In [803]:
data_20

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
C1021343,163.8,467.0,391.2,239.6,1.0,134698.04,5532.00,129166.04,16,19,...,2,1,2,2,2,31,0,1,0.016538,0.029610
C1040842,1.0,1.0,1.0,1.0,1.0,26387.17,26387.17,0.00,8,8,...,4,4,1,1,1,24,0,1,0.000038,0.000038
C1130989,60.0,60.0,60.0,60.0,1.0,250.26,250.26,0.00,10,10,...,6,5,1,1,1,25,0,0,0.239751,0.239751
C1219943,1892.0,1892.0,1892.0,1892.0,1.0,2951.10,2951.10,0.00,12,12,...,1,3,1,1,1,24,1,0,0.641117,0.641117
C1220223,148.0,148.0,148.0,148.0,1.0,95075.54,95075.54,0.00,17,17,...,0,2,1,1,1,34,0,1,0.001557,0.001557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C9099836,691.0,691.0,691.0,691.0,1.0,133067.23,133067.23,0.00,0,0,...,4,7,1,1,1,25,0,1,0.005193,0.005193
C9099877,222.0,222.0,222.0,222.0,1.0,96063.46,96063.46,0.00,12,12,...,3,9,1,1,1,20,0,1,0.002311,0.002311
C9099919,126.0,126.0,126.0,126.0,1.0,5559.75,5559.75,0.00,12,12,...,3,8,1,1,1,22,0,0,0.022663,0.022663
C9099941,50.0,50.0,50.0,50.0,1.0,35295.92,35295.92,0.00,21,21,...,6,8,1,1,1,21,0,1,0.001417,0.001417


In [804]:
data_20 = data_20.reset_index()
data_20.columns = col_list_str
data_20

,CustomerID,min_trans,max_trans,quant_75_trans,quant_25_trans,trans_val_nr,max_balance,min_balance,diff_balance,most_freq_hour,...,most_freq_weekday,most_freq_month,nr_diff_months,diff_locs,diff_dob,age_mean,nr_trans_1kp,nr_trans_10kp,tb_ratio_mean,tb_ratio_max
0,C1021343,163.8,467.0,391.2,239.6,1.0,134698.04,5532.00,129166.04,16,...,2,1,2,2,2,31,0,1,0.016538,0.029610
1,C1040842,1.0,1.0,1.0,1.0,1.0,26387.17,26387.17,0.00,8,...,4,4,1,1,1,24,0,1,0.000038,0.000038
2,C1130989,60.0,60.0,60.0,60.0,1.0,250.26,250.26,0.00,10,...,6,5,1,1,1,25,0,0,0.239751,0.239751
3,C1219943,1892.0,1892.0,1892.0,1892.0,1.0,2951.10,2951.10,0.00,12,...,1,3,1,1,1,24,1,0,0.641117,0.641117
4,C1220223,148.0,148.0,148.0,148.0,1.0,95075.54,95075.54,0.00,17,...,0,2,1,1,1,34,0,1,0.001557,0.001557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839076,C9099836,691.0,691.0,691.0,691.0,1.0,133067.23,133067.23,0.00,0,...,4,7,1,1,1,25,0,1,0.005193,0.005193
839077,C9099877,222.0,222.0,222.0,222.0,1.0,96063.46,96063.46,0.00,12,...,3,9,1,1,1,20,0,1,0.002311,0.002311
839078,C9099919,126.0,126.0,126.0,126.0,1.0,5559.75,5559.75,0.00,12,...,3,8,1,1,1,22,0,0,0.022663,0.022663
839079,C9099941,50.0,50.0,50.0,50.0,1.0,35295.92,35295.92,0.00,21,...,6,8,1,1,1,21,0,1,0.001417,0.001417


In [805]:
data_20.to_csv('twenty_params.csv', sep = ',', encoding = 'utf-8')

# 2.2 Choose your features (variables)!

### dim reduction